## Notes
<list>
<li>efective utilization and tardiness
<li>reward function
<li>Starting PointÖ
<li>2 recipes
<li>3 machines
<li>deadlines
</list>

### Questions:
<list>
<li> Is the queue of jobs part of the environment?
<li> Does the environment know which task of which job it is performing?
<li> Does a machine take the same time to work on all recipes? (if it knows how to make more than 1)
<li> Are the Jobs part of the Environment? Can they bee if they are constantly changing?
<li> Should i take the jobs one by one after ordering by priority and/or dealine (like tetris), or should i take into account some abount of  jobsin the queue to check if some of them can take place at the same time? that number can be an hyperparam
<li> should i set a limit to the amount of jobs i can read,so that the environment has finite dimensions 
</list>


In [176]:
class Machine:
    machine_type = "A" #A,B,C,D...
    knownRecipes = [] #R1,R2,R3... 
    trayCapacity = 10000
    status = 0    # 0-Free, 1-Busy - I am asuming you cannot open a machine that is workingt to add more trays
    activeRecipe = ""
    activeJobs = []
    startTime = ""
    
    def __init__(self, m_type, k_recipes, cap):
        self.machine_type = m_type
        self.knownRecipes=k_recipes
        self.trayCapacity = cap
        self.status = 0 #Free by default
        self.activeJobs= []
    
    def getKnownRecipes(self):
        return self.knownRecipes
    
    def getMachineType(self):
        return self.machine_type
    
    def getTrayCapacity(self):
        return self.trayCapacity
    
    def getStatus(self):
        return self.status
    
    def getTextStatus(self):
        if self.status == 0:
            return "Free"
        else:
            return "Busy"
    
    def updateKnownRecipes(self,new_recipes):
        self.knownRecipes=new_recipes
    
    def updateTrayCapacity(self,new_capacity):
        self.trayCapacity=new_capacity
    
    def assignJobs(self, jobs,recipe):
        assigned = self.setActiveRecipe(recipe)
        if assigned:
            for j in jobs:
                self.activeJobs.append(j)
                print(j)
                print(recipe)
                j.recipeInProgress(recipe)
            self.status = 1
            self.startTime = "now"
            return True
        else:
            return False
    
    def setActiveRecipe(self, recipe):
        if recipe in self.knownRecipes:
            self.activeRecipe = recipe
            return True
        else:
            return False
    
    def getActiveRecipe(self):
        return self.activeRecipe
    
    def getActiveJobsId(self):
        jobIds = ""
        for j in self.activeJobs:
            jobIds = jobIds + j.jobId + ", "
        return jobIds
    
    def resetMachine(self):
        self.status = 0    # 0-Free, 1-Busy - I am asuming you cannot open a machine that is workingt to add more trays
        self.activeRecipe = []
        self.activeJobs = []
    
    
    def __str__(self):
        return f'Type: {self.machine_type} \nKnown Recipes: {self.knownRecipes} \nTray Capacity: {self.trayCapacity} \nStatus: {self.getTextStatus()}\nWorking on recipe: {self.activeRecipe} for the following Jobs: {self.getActiveJobsId()}'
    


In [185]:
class Job:
    jobId = "ID-00000" #String for now, can be numeric
    recipes = [] #R1,R2,R3...
    quantity = 0
    deadline = "3000/01/01" #YYYY/MM/DD
    status = 0
    priority = 1
    r_pending =[]
    r_inProgress = []
    r_completed=[]
    
    def __init__(self, jobId, recipes, quantity,deadline, priority):
        self.r_inProgress = []
        self.r_completed=[]
        self.r_pending=[]
        self.jobId = jobId
        self.recipes=recipes
        self.r_pending = recipes.copy()
        self.quantity = quantity
        self.deadline = deadline
        self.priority = priority # 1-Normal, 2-Medium, 3-High
        self.status = 0   # 0-New, 1- In Progress, 2- Completed, 3- Cancelled?, 4-Error? (For internal use)

    
    def getRecipes(self):
        return self.recipes
    
    def getjobId(self):
        return self.jobId
    
    def getQuantity(self):
        return self.quantity
    
    def getDeadline(self):
        return self.deadline
    
    def getStatus(self):
        return self.status #might come in handy in the future
    
    def getPriority(self):
        return self.priority
    
    def getTextPriority(self):
        ref_table = {1:"Normal", 2:'Medium',3:"High"}
        try:
            return  ref_table[self.priority]
        except:
            return "Not Defined"
    
    def getTextStatus(self):
        ref_table = {0:"New", 1:'In Progress',2:"Completed",3:"Cancelled",4:"Error"}
        try:
            return  ref_table[self.status]
        except:
            return "Not Defined"
        
    def updateDeadline(self,new_deadline):
        self.deadline=new_deadline
    
    def updatePriority(self,new_priority):
        self.priority = new_priority
    
    def updateStatus(self,new_status):
        self.status = new_status
    #I didnt add an update recipes because it would add complexity in the future if someone can alter the recipes
    
    def recipeInProgress(self,recipe):
        print(recipe)
        print(self.r_pending)
        self.r_pending.remove(recipe)
        self.r_inProgress.append(recipe)
        self.status = 1
    
    def recipeCompleted(self, recipe):
        self.r_completed.append(recipe)
        self.r_inProgress.remove(recipe)
        if self.r_pending == [] and self.r_inProgress == []:
            self.status = 2
            
    def __str__(self):
        return f'Job ID: {self.jobId} \nRecipes: {self.recipes} \nQuantity: {self.quantity} \nDeadline: {self.deadline} \nPriority: {self.getTextPriority()} \nStatus: {self.getTextStatus()} \nIn Progress: {self.r_inProgress} \nCompleted: {100*len(self.r_completed)/len(self.recipes)} %'

In [186]:
# m1 = Machine(m_type="A", k_recipes=["A1","A2"], cap=10000)
# m2 = Machine(m_type="A", k_recipes=[3,2], cap=10000)
# m3 = Machine(m_type="A", k_recipes=[2], cap=10000)
# print(m1)
# print(m2)
# print(m3)
# print()
# j1= Job(jobId="ID-00000", recipes=["A1","A2"], quantity=3,deadline="2024/01/04", priority=2)
# j2= Job(jobId="ID-00001", recipes=["A2","A3"], quantity=10,deadline="2024/02/04", priority=3)
# j3= Job(jobId="ID-00002", recipes=["A3"], quantity=5,deadline="2023/12/04", priority=1)
# j1.recipeInProgress("A1")
# j1.recipeCompleted("A1")
# print(j1)
# print("/--------/")
# print(j2)
# print("/--------/")
# print(j3)
# print("/*********************************")
# def custom_key(item):
#     return (-item.getPriority(),item.getDeadline())

# js = sorted([j1,j2,j3],key=custom_key)
# for j in js:
#     print(j)


In [193]:
import gymnasium as gym
import numpy as np
from gymnasium import spaces
import matplotlib.pyplot as plt
import pandas as pd

#Base source: https://stable-baselines3.readthedocs.io/en/master/guide/custom_env.html


class FactoryEnv(gym.Env):
    """Custom Environment that follows gym interface."""

    metadata = {"render_modes": ["vector"]}

    def __init__(self, machines,jobs_path):
        super().__init__()
        # Define action and observation space
        # They must be gym.spaces objects
        self.jobs = []
        self.pickUpJobs(jobs_path)
        self.machines = machines
        
        self.action_space = spaces.Discrete(len(self.machines)) # possible machines to allocate jobs
        self.observation_space = spaces.Box(low=0, high=1, shape=(len(self.jobs), len(self.machines)), dtype=int)
        #PARAMETERS:
            # low (Union[SupportsFloat, np.ndarray]) – Lower bounds of the intervals.
            # high (Union[SupportsFloat, np.ndarray]) – Upper bounds of the intervals.
            # shape (Optional[Sequence[int]]) – The shape is inferred from the shape of low or high np.ndarray`s with `low and high scalars defaulting to a shape of (1,)
            # dtype – The dtype of the elements of the space. If this is an integer type, the Box is essentially a discrete space.
            # seed – Optionally, you can use this argument to seed the RNG that is used to sample from the space.
       
    def pickUpJobs(self,path):
        jobs_df = pd.read_csv(path)
        for i,r in jobs_df.iterrows():
            r_rec = r["Recipes"].replace("'","")
            r_rec = r_rec.split(',')
            j = Job(jobId=r["Id"], recipes=r_rec, quantity=r["Quantity"],deadline=r["Deadline"], priority=r["Priority"])
            self.jobs.append(j)
    
    def custom_key(item): #this will create a pair (priority-deadline) for sorting the jobs
        return (-item.getPriority(),item.getDeadline())

    def sortJobs(self):
        self.jobs = sorted(self.jobs,key=custom_key)

    def computeStepReward(self):
#         machine_utilization = 0
#         spare_time = 0
#         past_due = 0
#         queue_len = 0
#         return w1*spare_time + w2*machine_utilization + w3*past_due + w4*queue_len
        return -1
    
    def getJobs(self):
        return self.jobs
    
    def step(self, action, jobs, recipe): #or (action,job,recipe) tuple = action
        #Return values definition
        reward = 0
        info=""
        terminated = False
        truncated = False
        
        #my actions will be more complicated since will involve assigning one or more parts of jobs to machines.
        #example: a1 = assign R1(J1) to M1
        #         a1 = assign R1(J1) and R1(J3) to M1
        if action not in range(len(self.machines)) or self.machines[action].getStatus == 1 or recipe not in self.machines[action].getKnownRecipes():
            #truncated = True
            info = "Invalid Machine or Invalid Recipe for Machine the moment"
            reward = -10 #penalization for taking a wrong action
        
        #still pending reward calculations
        reward = self.computeStepReward() #to be placed
        
        self.machines[action].assignJobs(jobs,recipe)
        #still have pending the time management of the processes
        
#         if self.elapsed_time > self.max_steps:
#             terminated=True
        
        observation = []
        for m in self.machines:
            observation.append(m)
            
        return observation, reward, terminated, truncated, info

    def reset(self, seed=None, options=None):
        for m in self.machines:
            m.resetMachine()
        return self.machines.copy()
        #return observation, info

    def render(self):
        print("/********MACHINES********/")
        for m in self.machines:
            print(m)
            print("/-------------------------/")
        print("/**********JOBS**********/")
        for j in self.jobs:
            print(j)
            print("/-------------------------/")
        
        return (self.jobs,self.machines)


    def close(self):
        self.reset()

In [191]:
m1 = Machine(m_type="A", k_recipes=["A1","A2"], cap=10000)
m2 = Machine(m_type="A", k_recipes=["A2"], cap=10000)

machines = [m1,m2]
env = FactoryEnv(machines=machines,jobs_path ='jobs.csv')
p_jobs = env.getJobs()
o,r,t,tr,i = env.step(0,p_job[0],"A1")

In [192]:
env.render()

/********MACHINES********/
Type: A 
Known Recipes: ['A1', 'A2'] 
Tray Capacity: 10000 
Status: Free
Working on recipe:  for the following Jobs: 
/-------------------------/
Type: A 
Known Recipes: ['A2'] 
Tray Capacity: 10000 
Status: Free
Working on recipe:  for the following Jobs: 
/-------------------------/
/**********JOBS**********/
Job ID: ID-0000 
Recipes: ['A1', 'A2'] 
Quantity: 3 
Deadline: 24-04-01 
Priority: Medium 
Status: New 
In Progress: [] 
Completed: 0.0 %
/-------------------------/
Job ID: ID-0001 
Recipes: ['A2'] 
Quantity: 10 
Deadline: 24-04-14 
Priority: Normal 
Status: New 
In Progress: [] 
Completed: 0.0 %
/-------------------------/
Job ID: ID-0002 
Recipes: ['A1', 'A2'] 
Quantity: 5 
Deadline: 24-04-02 
Priority: High 
Status: New 
In Progress: [] 
Completed: 0.0 %
/-------------------------/


([<__main__.Job at 0x2c68459c550>,
 [<__main__.Machine at 0x2c68452e510>, <__main__.Machine at 0x2c68452ef10>])

False